In [1]:
%load_ext autoreload
%autoreload 2
from astropy.io import fits
import numpy as np
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
%matplotlib inline
import matplotlib.pyplot as plt
from photutils import CircularAnnulus, CircularAperture, RectangularAperture, DAOStarFinder, Background2D
from photutils import aperture_photometry,  deblend_sources, detect_sources, detect_threshold, source_properties
import os 
from ginga.util import zscale

from astropy.wcs import WCS
from wfc3_photometry import photometry_tools

import cosmics

In [2]:
test_dir = '/Users/cshanahan/Desktop/clean_desktop/WD_acs/data/dgq_08/F625W/'
test_file = '/Users/cshanahan/Desktop/clean_desktop/WD_acs/data/dgq_08/F625W/jdgq08030_drc.fits'

data = fits.open(test_file)[1].data
hdr = fits.open(test_file)[1].header
ctx = fits.open(test_file)[3].data

ndrizim = fits.getval(test_file, 'ndrizim')

In [3]:
def detect_sources_segm(data, threshold, npixels, kernel_fwhm = 1.8):
    """Use segmentation map to detect sources in image."""

    sigma = kernel_fwhm * gaussian_fwhm_to_sigma
    kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
    kernel.normalize()
    segm = detect_sources(data, threshold=threshold, npixels=npixels, filter_kernel=kernel)
    segm_cat = source_properties(data, segm).to_table()
    return(segm, segm_cat, kernel)
    

In [4]:
def filter_bad_sources(ctx_img, xpos, ypos, frac_cover_min, n_exp_total, val = True):

    #boolean array indicating if that pixel is covered by ANY image.
    coverage_arr = np.nan_to_num(ctx_img).astype(bool).astype(int)
    low_coverage_flags = []

    for i, xx in enumerate(xpos):
        if val == False:
            yy = ypos[i]
        else:
            xx, yy = xx.value, ypos[i].value
        ap = RectangularAperture((xx, yy), w=100, h=100)
        ap_phot = aperture_photometry(coverage_arr, ap)
        summ = ap_phot['aperture_sum'][0]
        if summ/ap.area() > frac_cover_min:
            low_coverage_flags.append(1)
        else:
            low_coverage_flags.append(0)

    return low_coverage_flags

In [5]:
def write_source_catalog(source_catalog, output_file_path, wcss):
    """Writes catalog of output sources as csv for use with ds9."""
    with open(output_file_path, 'w') as f:
        f.write('x, y\n')
        for i, val in enumerate(source_catalog):
            ra, dec = wcss.wcs_pix2world(source_catalog['xcentroid'][i].value, source_catalog['ycentroid'][i].value, 1)
            fwhmm = source_catalog['fwhm'][i]
            f.write(str(ra)+', '+str(dec)+', '+str(fwhmm)+'\n')

In [6]:
def add_fwhm_to_photcat(data, photcat):
	"""Uses wfc3photometry tool to fit a radial profile to every object in
	in the photometry table and adds the FWHM as a column."""

	fwhms = []
	for i, val in enumerate(photcat):
		x, y = photcat['xcentroid'][i].value, photcat['ycentroid'][i].value
		rad_prof = photometry_tools.rad_prof.RadialProfile(x, y, data)
		fwhms.append(rad_prof.fwhm)
	photcat['fwhm'] = fwhms
	return photcat

## Segmentation map

In [7]:
#try using background image
back_img = Background2D(data, box_size = 50)

/Users/cshanahan/miniconda3/envs/astroconda36/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:165: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/Users/cshanahan/miniconda3/envs/astroconda36/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:166: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value


In [8]:
threshold_img = detect_threshold(data, snr=2, background=back_img.background)
segm, segm_cat, kernel = detect_sources_segm(data, threshold_img, 100, kernel_fwhm = 2.5)

bad_sources = filter_bad_sources(ctx, segm_cat['xcentroid'], segm_cat['ycentroid'], 0.98, ndrizim)
segm_cat['bad_sources'] = bad_sources
segm_cat = segm_cat[segm_cat['bad_sources'] == 1]
wcss = WCS(fits.open(test_file)[1].header)

#fit fwhm
segm_cat = add_fwhm_to_photcat(data, segm_cat)

write_source_catalog(segm_cat, test_dir + 'sources_segm.csv', wcss)

/Users/cshanahan/miniconda3/envs/astroconda36/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:165: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/Users/cshanahan/miniconda3/envs/astroconda36/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:166: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value
/Users/cshanahan/miniconda3/envs/astroconda36/lib/python3.6/site-packages/photutils/segmentation/detect.py:128: RuntimeWarning: invalid value encountered in greater
  check_normalization=True) > threshold)


Optimal parameters not found: The maximum number of function evaluations is exceeded.
`x0` is infeasible.


In [ ]:
#display
os.system('ds9 {} -zscale -catalog import tsv {}sources_segm.csv'.format(test_file, test_dir))

## Deblend sources

In [ ]:
segm_deblend = deblend_sources(data, segm, npixels=75, filter_kernel=kernel, nlevels=3, 
                                   contrast=0.5)
segm_deblend_cat = source_properties(data, segm_deblend).to_table()

#fit fwhm
segm_deblend_cat = add_fwhm_to_photcat(data, segm_deblend_cat)

bad_sources = filter_bad_sources(ctx, segm_deblend_cat['xcentroid'], segm_deblend_cat['ycentroid'], 0.98, ndrizim)
segm_deblend_cat['bad_sources'] = bad_sources
segm_deblend_cat = segm_deblend_cat[segm_deblend_cat['bad_sources'] == 1]

segm_deblend_cat = segm_deblend_cat[segm_deblend_cat['fwhm'] > 1.8]
write_source_catalog(segm_deblend_cat, test_dir + 'sources_segm_deblended.csv', wcss)

print(len(segm_cat), len(segm_deblend_cat))

In [ ]:
#display
os.system('ds9 {0} -zscale -catalog import tsv {1}sources_segm_deblended.csv -catalog psky icrs'.format(test_file, test_dir))


In [ ]:

write_source_catalog(segm_deblend_cat_fwhm_filtered, test_dir + 'sources_segm_deblended_fwhm.csv', wcss)

In [ ]:
os.system('ds9 {0} -zscale -catalog import tsv {1}sources_segm_deblended_fwhm.csv -catalog psky icrs'.format(test_file, test_dir))
-zscale -catalog import tsv sources_segm_deblended_fwhm.csv -catalog psky icrs


## try lacosmic

In [ ]:
# Build the object :
c = cosmics.cosmicsimage(data, gain=1.5, readnoise=3.0, sigclip = 10, sigfrac = 0.4, objlim = 2, verbose=False)
# cosmics module is very verbose, if you wish to turn off check out cosmics.py
c.run(maxiter=5, verbose=False)
print('DONE')

cr_mask = c.mask

## detect point sources with daofind

In [ ]:
daofind = DAOStarFinder(threshold=10, fwhm=1.8)
daofind_sources = daofind(data)

bad_sources = filter_bad_sources(ctx, daofind_sources['xcentroid'], daofind_sources['ycentroid'], 0.98, ndrizim, val=False)
daofind_sources['bad_sources'] = bad_sources
daofind_sources = daofind_sources[daofind_sources['bad_sources'] == 1]

with open(test_dir + 'sources_daofind.csv', 'w') as f:
    f.write('x, y\n')
    for i, val in enumerate(daofind_sources):
        f.write(str(daofind_sources['xcentroid'][i])+', '+str(daofind_sources['ycentroid'][i])+'\n')


In [ ]:
#display
os.system('ds9 {0} -zscale -catalog import tsv {1}sources_daofind.csv -catalog psky image'.format(test_file, test_dir))


In [ ]:
plt.figure(figsize = (20, 20))
z1, z2 = zscale.zscale(data)
plt.imshow(data, origin='lower', vmin=z1, vmax=z2)
plt.scatter(daofind_sources['xcentroid'], daofind_sources['ycentroid'], facecolor = 'None', edgecolor = 'r')
plt.scatter(segm_deblend_cat['xcentroid'], segm_deblend_cat['ycentroid'], facecolor = 'None', edgecolor = 'g', s = 20)